In [ ]:
import numpy as np
import pymc as pm
import pandas as pd
import arviz as az
import matplotlib
import matplotlib.pyplot as plt
import os
import math
import pickle
import ast
import xarray as xr

In [ ]:
#load and unpack data
with open("Data_A\\ReadyData_Synth_A.pkl", "rb") as f:
    data_dict = pickle.load(f)

cov_mat = data_dict['cov_mat']
grp_idx = data_dict['grp_idx']
obs_data = data_dict['resp']
params_fixed = data_dict['params_fixed']

In [ ]:
exec(open("Build_Model_A.py").read())

Add PSE and JND

In [ ]:
with model_A:
    PSE = pm.Deterministic("PSE", (-beta_vec[0] + pm.math.log( (1-2*gam_h) / (1-2*gam_l) ))/ beta_vec[1] , dims=("groups",))
    JND = pm.Deterministic("JND", (pm.math.log( ((3-4*gam_h)*(3-4*gam_l)) / ((1-4*gam_h)*(1-4*gam_l)) )) / (2*beta_vec[1]) , dims=("groups",))

In [ ]:
with model_A:
    trace = pm.sample(return_inferencedata=True, chains = 4, cores=1, progressbar=True, idata_kwargs={"log_likelihood": True})   
print("FINISHED SAMPLING!")

Look at r_hats and effective sample sizes, looks good

In [ ]:
az.summary(trace, var_names = ['beta_vec', 'gam_h', 'gam_l', 'PSE', 'JND'])

Look at joint posteriors for parameters. Some identifiability issues.

In [ ]:
grp_num = 0
grp_choice = coords['groups'][grp_num]

param_bases = ['gam_h ', 'gam_l ','beta_vec b0, ','beta_vec b1, ']
ref_vals = {}

for grp_num, grp_choice in enumerate(coords['groups']):
    temp_dict = {}
    for par_num, par_base in enumerate(param_bases):
        temp_dict[par_base+grp_choice] = params_fixed[par_num][grp_num]
    ref_vals[grp_choice] = temp_dict

In [ ]:
for grp_num, grp_choice in enumerate(coords['groups']):
     az.plot_pair(trace, var_names=['gam_h', 'gam_l', 'beta_vec'], 
             coords = {'betas': ["b0", "b1"], 'groups': [grp_choice]}, 
             reference_values = ref_vals[grp_choice],
                       reference_values_kwargs=dict(marker="o", color="red", markersize=8),
             kind = 'kde', marginals=True)

Look at joint posteriors for PSE and JND show less identifiability isses.

In [ ]:

param_bases2 = ['gam_h ', 'gam_l ']
ref_vals2 = {}

for grp_num, grp_choice in enumerate(coords['groups']):
    temp_dict = {}
    for par_num, par_base in enumerate(param_bases2):
        temp_dict[par_base+grp_choice] = params_fixed[par_num][grp_num]
    ref_vals2[grp_choice] = temp_dict
    PSE = (-params_fixed[2][grp_num] + np.log( (1-2*params_fixed[0][grp_num]) / (1-2*params_fixed[1][grp_num]) ))/ params_fixed[3][grp_num]
    JND = np.log( ((3-4*params_fixed[0][grp_num])*(3-4*params_fixed[1][grp_num])) / 
                 ((1-4*params_fixed[0][grp_num])*(1-4*params_fixed[1][grp_num])) ) / (2*params_fixed[3][grp_num])
    ref_vals2[grp_choice]['PSE '+grp_choice] = PSE
    ref_vals2[grp_choice]['JND '+grp_choice] = JND

for grp_num, grp_choice in enumerate(coords['groups']):
    az.plot_pair(trace, var_names=['gam_h', 'gam_l', 'PSE', 'JND'], 
             coords = {'groups': [grp_choice]},  
             reference_values = ref_vals2[grp_choice],
                       reference_values_kwargs=dict(marker="o", color="red", markersize=8),
             kind = 'kde', marginals=True)

Look at traceplots: everything seems good

In [ ]:
az.plot_trace(trace, var_names=('gam_h', 'gam_l', 'beta_vec'), coords = {
    'groups': ['left_bi'],
    'betas': ["b0", "b1"]}, compact=False,  backend_kwargs={"constrained_layout": True})

In [ ]:
param_samps = trace.posterior[['beta_vec', 'gam_h', 'gam_l']]

In [ ]:
gam_h_samps = {}
gam_l_samps = {}
beta_0_samps = {}
beta_1_samps = {}


for grp in ["left_bi","left_uni","right_bi","right_uni"]:
    gam_h_samps[grp] = param_samps['gam_h'].sel(groups = grp).values.flatten()
    gam_l_samps[grp] = param_samps['gam_l'].sel(groups = grp).values.flatten()
    beta_0_samps[grp] = param_samps['beta_vec'].sel(groups = grp, betas='b0').values.flatten()
    beta_1_samps[grp] = param_samps['beta_vec'].sel(groups = grp, betas='b1').values.flatten()

In [ ]:
def psychfunc(params, X):
    """
    Psychometric function with lapses

    Parameters:
    params : [gamma, lambda_, beta0, beta1]
    X : Stimulus amplitude level

    Returns:
    probability of guess "high"

    """
    X = np.asarray(X)
    gam_h, gam_l, beta0, beta1 = params
    logistic = 1 / (1 + np.exp(-(beta0 + beta1 * X)))
    return gam_h + (1 - gam_h - gam_l) * logistic

In [ ]:
#frquencies for each level
freq_df = pd.DataFrame({'stim': cov_mat[:,1], 'grp_idx': grp_idx, 'obs_data': obs_data})
freqs = pd.pivot_table(
    freq_df, 
    values='obs_data',
    index='stim',
    columns='grp_idx',
    aggfunc='mean'
)

In [ ]:
xfit = np.linspace(-1.6,1.6,500)
y_samples = {}
hdis = {}
rec_params = {}
yrec = {}
fixed = np.array(params_fixed)

for grp_i, grp in enumerate(['left_uni','left_bi','right_uni','right_bi']):
    y_samples[grp] = np.array([psychfunc([gam_h,gam_l,beta_0,beta_1], xfit) 
                          for gam_h,gam_l,beta_0,beta_1 in zip(
                              gam_h_samps[grp], gam_l_samps[grp], beta_0_samps[grp], beta_1_samps[grp])])
    hdis[grp] = az.hdi(y_samples[grp], hdi_prob=0.95)
    rec_params[grp] = np.mean(np.array([gam_h_samps[grp], gam_l_samps[grp], beta_0_samps[grp], beta_1_samps[grp]]), axis = 1)
    
    
    yrec[grp] = psychfunc(rec_params[grp], xfit)
    
    plt.plot(xfit,yrec[grp],label='Recovered Curve',color='green')
    plt.plot(xfit, psychfunc(fixed[:,grp_i], xfit), label='Original Curve',color='red')
    plt.fill_between(xfit, hdis[grp][:, 0], hdis[grp][:, 1], color='green', alpha=0.3, label='95% HDI')
    plt.scatter(np.array(freqs.index),np.array(freqs[grp_i]),label='Data', color = 'red')
    plt.title(grp)
    plt.xlabel('Stimulus Amplitude')
    plt.legend(loc='upper left', fontsize=9.5)
    plt.show()   

In [ ]:
with model_A:
    pm.sample_posterior_predictive(trace,extend_inferencedata=True)

In [ ]:
az.plot_ppc(trace, num_pp_samples=100)

In [ ]:
true_probs = np.zeros([8,4])
pred_probs = np.zeros([8,4])
fixed = np.array(params_fixed)
for grp_i, grp in enumerate(['left_uni','left_bi','right_uni','right_bi']):
    true_probs[:,grp_i] = psychfunc(fixed[:,grp_i],np.array(freqs.index))
    pred_probs[:,grp_i] = psychfunc(rec_params[grp],np.array(freqs.index))

In [ ]:
true_prop = np.array(freqs)

We can see that the model does a better job recovering the original response probabilities than the frequencies of the data.

In [ ]:
RMSE = np.sqrt(np.mean(np.array(freq_df['pred_prob']-freq_df['true_prob'])**2))
print(RMSE)

In [ ]:
np.sqrt(np.mean(np.array(freq_df['true_prop']-freq_df['true_prob'])**2))

In [ ]:
az.loo(trace)